In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy

### Pré-processing

In [2]:
data_train = pd.read_csv('../../data/archive/train.csv')
data_test = pd.read_csv('../../data/archive/test.csv')

In [3]:
data_train

,Unnamed: 0,film-url,review,polarity
0,0,http://www.allocine.fr/film/fichefilm-135259/c...,Si vous cherchez du cinéma abrutissant à tous ...,0
1,1,http://www.allocine.fr/film/fichefilm-172430/c...,"Trash, re-trash et re-re-trash...! Une horreur...",0
2,2,http://www.allocine.fr/film/fichefilm-15105/cr...,"Et si, dans les 5 premières minutes du film, l...",0
3,3,http://www.allocine.fr/film/fichefilm-188629/c...,Mon dieu ! Quelle métaphore filée ! Je suis ab...,0
4,4,http://www.allocine.fr/film/fichefilm-23514/cr...,"Premier film de la saga Kozure Okami, ""Le Sabr...",1
...,...,...,...,...
159995,159995,http://www.allocine.fr/film/fichefilm-132387/c...,Un rythme bien trop lent et un Ashton Kutcher ...,0
159996,159996,http://www.allocine.fr/film/fichefilm-53313/cr...,Monsieur Duchovny vous êtes aussi piètre acteu...,0
159997,159997,http://www.allocine.fr/film/fichefilm-248258/c...,Complètement différent des films de la série C...,1
159998,159998,http://www.allocine.fr/film/fichefilm-268731/c...,Alors franchement pour le moment c'est le meil...,1


In [4]:
nlp = spacy.load("fr_core_news_sm")
french_stop_words = nlp.Defaults.stop_words

# Initialisation de CountVectorizer avec les stop words
vectorizer = CountVectorizer(stop_words=list(french_stop_words))

# Application de CountVectorizer sur la colonne 'review'
X_train = vectorizer.fit_transform(data_train['review'])
X_test = vectorizer.transform(data_test['review'])

y_train = data_train["polarity"]
y_test = data_test["polarity"]

print("Taille de la matrice de features pour train :", X_train.shape)
print("Taille de la matrice de features pour test :", X_test.shape)
print("Les premiers mots uniques extraits :", vectorizer.get_feature_names_out()[:30])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


Taille de la matrice de features pour train : (160000, 152189)
Taille de la matrice de features pour test : (20000, 152189)
Les premiers mots uniques extraits : ['00' '000' '000001ct' '0001' '000m' '000mots' '000volts' '001'
 '003023_21708_2' '005' '006' '007' '01' '015' '01h15' '01h30' '01h37'
 '01h40' '01h45' '01h49' '01h50' '01h57' '01min' '02' '02h10' '02h20'
 '02h45' '03' '031119' '04']


### Conception du modèle

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, accuracy_score, recall_score

model_lr = LogisticRegression(max_iter = 10000)
model_lr.fit(X_train, y_train)

prediction = model_lr.predict(X_test)

precision = precision_score(y_test, prediction)
accuracy = accuracy_score(y_test, prediction)
recall = recall_score(y_test, prediction)

print("La précision est de :", precision)
print("L'accuracy est de :", accuracy)
print("Le recall est de :", recall)

La précision est de : 0.9103319888302823
L'accuracy est de : 0.91715
Le recall est de : 0.9176396997497915


The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The accuracy is the ratio of (values right predicted) / (number of total values)

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

Here for our case the metric to proritize seems to be the accuracy.

### Création du pipeline scikit-learn

In [6]:
from sklearn.pipeline import Pipeline

estimators = [('vectorizer', CountVectorizer(stop_words=list(french_stop_words))), 
              ('model', LogisticRegression(max_iter = 10000))]
pipe = Pipeline(estimators)

pipe.fit(data_train["review"], y_train)
pipe_prediction = pipe.predict(data_test["review"])

pipe_accuracy = accuracy_score(y_test, pipe_prediction)

print("L'accuracy de la pipeline est de :", pipe_accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


L'accuracy de la pipeline est de : 0.91715


### Expérimentation de différents modèles et hyperparamètres

La régression logistique est un modèle de classification robuste et efficace pour les tâches de classification binaire. Elle est particulièrement bien pour des jeux de données où les classes sont bien séparées. En fonctionnant par probabilité qu'un avis soit positif ou négatif, elle est interprétable et rapide à entraîner, ce qui la rend idéale pour des tâches de classification de texte comme prédire la polarité des avis.

In [7]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

Best parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best accuracy: 0.9152687499999999


Le MLP, un réseau de neurones, est une solution plus puissante. Il peut capturer des relations non linéaires entre les caractéristiques, ce qui le rend performant sur des tâches complexes de classification, comme la détection de la polarité dans des avis qui peuvent contenir des nuances subtiles dans leur langage.

In [ ]:
from sklearn.neural_network import MLPClassifier

param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'learning_rate': ['constant', 'adaptive']
}
grid_search_mlp = GridSearchCV(MLPClassifier(max_iter=1000), param_grid_mlp, cv=5, scoring='accuracy')
grid_search_mlp.fit(X_train, y_train)

print("Best parameters for MLP:", grid_search_mlp.best_params_)
print("Best accuracy for MLP:", grid_search_mlp.best_score_)

La SVC est une méthode efficace pour les jeux de données avec des séparation claires entre les classes. Elle peut gérer les jeux de données avec des relations complexes grâce à l'utilisation de noyaux. Dans le contexte de la polarité des avis, la SVC est capable de classifier efficacement même lorsque les données sont non linéaires ou lorsqu'elles sont bruitées, ce qui la rend solide pour cette tâche.

In [ ]:
from sklearn.svm import SVC

param_grid_svc = {'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf']}
grid_search_svc = GridSearchCV(SVC(), param_grid_svc, cv=5, scoring='accuracy')
grid_search_svc.fit(X_train, y_train)

print("Best parameters for SVC:", grid_search_svc.best_params_)
print("Best accuracy for SVC:", grid_search_svc.best_score_)